In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
( train_ds , test_ds ), ds_info = tfds.load('mnist', split =['train','test'], as_supervised=True, with_info=True)

In [ ]:
batchsize = 128

In [ ]:
def prepare_mnist_data(ds):
  # Transform the dataset by mapping each element (feature_dict) to a tuple containing the "image" and "label" components.
  #ds = ds.map(lambda feature_dict: (feature_dict["image"], feature_dict["label"]))

  # Reshape the images by mapping each element (image, label) to a tuple with the image reshaped to a 1D vector and the label.
  ds = ds.map(lambda image, label: (tf.reshape(image, (-1,)), label))

  # Scale and normalize the images to the range [-1, 1] by mapping each element (image, label) to a tuple.
  # This is done by casting the image to float32 and dividing by 128, then subtracting 1.
  ds = ds.map(lambda image, label: ((tf.cast(image, tf.float32)/128)-1, label))

  # One-hot encode the labels by mapping each element (image, label) to a tuple where the label is one-hot encoded with depth 10.
  ds = ds.map(lambda image, label: (image, tf.one_hot(label, depth=10)))

  # Shuffle the dataset with a buffer size of 1024 and batch the shuffled data into batches of size 256.
  ds = ds.shuffle(1024).batch(batchsize)

  # Prefetch the data to improve performance by overlapping data preprocessing and model execution.
  ds = ds.prefetch(4)
  return ds

In [ ]:
train_dataset = train_ds.apply(prepare_mnist_data)
test_dataset = test_ds.apply(prepare_mnist_data)

batchsize = 500

train_dataset1 = train_ds.apply(prepare_mnist_data)
test_dataset1 = test_ds.apply(prepare_mnist_data)

In [ ]:
#Define the MLP Model
class MLP_Model(tf.keras.Model):
  def __init__(self, layer_sizes, output_size=10):
    super().__init__()
    self.mlp_layers = []

    for layer_size in layer_sizes:
      new_layer = tf.keras.layers.Dense(units=layer_size, activation="sigmoid")
      self.mlp_layers.append(new_layer)
    self.output_layer = tf.keras.layers.Dense(units=output_size, activation="softmax")

  def call(self, x):
    for layer in self.mlp_layers:
      x = layer(x)
    y = self.output_layer(x)
    return y

In [ ]:
def train_mnist_model(train_ds, model, loss_func, optimizer, num_epochs):
  # Initialize lists to store training and test loss and accuracy
  train_loss_list = []
  train_accuracy_list = []
  # Initialize the accuracy metrics
  train_accuracy_metric = tf.keras.metrics.Accuracy()

  for epoch in range(num_epochs):
    # Training
    total_train_loss = 0.0
    train_accuracy_metric.reset_states()

    for inputs, labels in train_ds:
        with tf.GradientTape() as tape:
            outputs = model(inputs)
            train_loss = loss_func(labels, outputs)

        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        total_train_loss += train_loss.numpy()
        train_accuracy_metric.update_state(tf.argmax(labels, axis=1), tf.argmax(outputs, axis=1))

    average_train_loss = total_train_loss / len(train_ds)
    train_accuracy = train_accuracy_metric.result().numpy()

    # Print or log the values
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')


    # Store values for plotting
    train_loss_list.append(average_train_loss)
    train_accuracy_list.append(train_accuracy)

  return (train_loss_list, train_accuracy_list)

In [ ]:
def test_mnist_model(test_ds, model, loss_func, optimizer, num_epochs):
  test_loss_list = []
  test_accuracy_list = []

  # Initialize the accuracy metrics
  test_accuracy_metric = tf.keras.metrics.Accuracy()

  for epoch in range(num_epochs):
    # Testing
    total_test_loss = 0.0
    test_accuracy_metric.reset_states()

    for test_inputs, test_labels in test_ds:
        test_outputs = model(test_inputs)
        test_loss = loss_func(test_labels, test_outputs)

        total_test_loss += test_loss.numpy()
        test_accuracy_metric.update_state(tf.argmax(test_labels, axis=1), tf.argmax(test_outputs, axis=1))

    average_test_loss = total_test_loss / len(test_ds)
    test_accuracy = test_accuracy_metric.result().numpy()

    # Print or log the values
    print(f'Epoch {epoch + 1}/{num_epochs}, Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    # Store values for plotting
    test_loss_list.append(average_test_loss)
    test_accuracy_list.append(test_accuracy)

  return (test_loss_list, test_accuracy_list)



In [ ]:
def train_test_mnist_model(train_ds, test_ds, model, loss_func, optimizer, num_epochs):
  # Initialize lists to store training and test loss and accuracy
  train_loss_list = []
  train_accuracy_list = []
  test_loss_list = []
  test_accuracy_list = []

  # Initialize the accuracy metrics
  train_accuracy_metric = tf.keras.metrics.Accuracy()
  test_accuracy_metric = tf.keras.metrics.Accuracy()

  # Training loop
  num_epochs = 10  # You can adjust this based on your needs

  for epoch in range(num_epochs):
    # Training
    total_train_loss = 0.0
    train_accuracy_metric.reset_states()

    for inputs, labels in train_ds:
        with tf.GradientTape() as tape:
            outputs = model(inputs)
            train_loss = loss_func(labels, outputs)

        gradients = tape.gradient(train_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        total_train_loss += train_loss.numpy()
        train_accuracy_metric.update_state(tf.argmax(labels, axis=1), tf.argmax(outputs, axis=1))

    average_train_loss = total_train_loss / len(train_ds)
    train_accuracy = train_accuracy_metric.result().numpy()

    # Testing
    total_test_loss = 0.0
    test_accuracy_metric.reset_states()

    for test_inputs, test_labels in test_ds:
        test_outputs = model(test_inputs)
        test_loss = loss_func(test_labels, test_outputs)

        total_test_loss += test_loss.numpy()
        test_accuracy_metric.update_state(tf.argmax(test_labels, axis=1), tf.argmax(test_outputs, axis=1))

    average_test_loss = total_test_loss / len(test_ds)
    test_accuracy = test_accuracy_metric.result().numpy()

    # Print or log the values
    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {average_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

    # Store values for plotting
    train_loss_list.append(average_train_loss)
    train_accuracy_list.append(train_accuracy)
    test_loss_list.append(average_test_loss)
    test_accuracy_list.append(test_accuracy)

  return (train_loss_list, train_accuracy_list, test_loss_list, test_accuracy_list)


In [ ]:
model = MLP_Model([256,256])
model1 = MLP_Model([64,32])
model2 = MLP_Model([25,25])
model3 = MLP_Model([20])
model4 = MLP_Model([1])

cce = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.1)
ds = train_dataset
test_ds = test_dataset
num_epochs = 8

#Diffrent models with diffrent layers
print("Model 0, SDG")
(train_loss_list0_0, train_accuracy_list0_0, test_loss_list0_0, test_accuracy_list0_0) = train_test_mnist_model(train_dataset, test_dataset, model, cce, optimizer, num_epochs)
print("Model 0, ADAM")
(train_loss_list0_1, train_accuracy_list0_1, test_loss_list0_1, test_accuracy_list0_1) = train_test_mnist_model(train_dataset, test_dataset, model, cce, optimizer, num_epochs)
print("Model 0, ARMSProp")
(train_loss_list0_2, train_accuracy_list0_2, test_loss_list0_2, test_accuracy_list0_2) = train_test_mnist_model(train_dataset, test_dataset, model, cce, optimizer, num_epochs)
print("Model 1, SDG")
(train_loss_list1_0, train_accuracy_list1_0, test_loss_list1_0, test_accuracy_list1_0) = train_test_mnist_model(train_dataset, test_dataset, model1, cce, optimizer, num_epochs)
print("Model 1, ADAM")
(train_loss_list1_1, train_accuracy_list1_1, test_loss_list1_1, test_accuracy_list1_1) = train_test_mnist_model(train_dataset, test_dataset, model1, cce, optimizer, num_epochs)
print("Model 1, ARMSProp")
(train_loss_list1_2, train_accuracy_list1_2, test_loss_list1_2, test_accuracy_list1_2) = train_test_mnist_model(train_dataset, test_dataset, model1, cce, optimizer, num_epochs)
print("Model 2, SDG")
(train_loss_list2_0, train_accuracy_list2_0, test_loss_list2_0, test_accuracy_list2_0) = train_test_mnist_model(train_dataset, test_dataset, model2, cce, optimizer, num_epochs)
print("Model 2, ADAM")
(train_loss_list2_1, train_accuracy_list2_1, test_loss_list2_1, test_accuracy_list2_1) = train_test_mnist_model(train_dataset, test_dataset, model2, cce, optimizer, num_epochs)
print("Model 2, ARMSProp")
(train_loss_list2_2, train_accuracy_list2_2, test_loss_list2_2, test_accuracy_list2_2) = train_test_mnist_model(train_dataset, test_dataset, model2, cce, optimizer, num_epochs)
print("Model 3, SDG")
(train_loss_list3_0, train_accuracy_list3_0, test_loss_list3_0, test_accuracy_list3_0) = train_test_mnist_model(train_dataset, test_dataset, model3, cce, optimizer, num_epochs)
print("Model 3, ADAM")
(train_loss_list3_1, train_accuracy_list3_1, test_loss_list3_1, test_accuracy_list3_1) = train_test_mnist_model(train_dataset, test_dataset, model3, cce, optimizer, num_epochs)
print("Model 3, ARMSProp")
(train_loss_list3_2, train_accuracy_list3_2, test_loss_list3_2, test_accuracy_list3_2) = train_test_mnist_model(train_dataset, test_dataset, model3, cce, optimizer, num_epochs)
print("Model 4, SDG")
(train_loss_list4_0, train_accuracy_list4_0, test_loss_list4_0, test_accuracy_list4_0) = train_test_mnist_model(train_dataset, test_dataset, model4, cce, optimizer, num_epochs)
print("Model 4, ADAM")
(train_loss_list4_1, train_accuracy_list4_1, test_loss_list4_1, test_accuracy_list4_1) = train_test_mnist_model(train_dataset, test_dataset, model4, cce, optimizer, num_epochs)
print("Model 4, ARMSProp")
(train_loss_list4_2, train_accuracy_list4_2, test_loss_list4_2, test_accuracy_list4_2) = train_test_mnist_model(train_dataset, test_dataset, model4, cce, optimizer, num_epochs)



In [ ]:
def visualization(train_losses, train_accuracies, test_losses, test_accuracies, name):
    """Visualizes accuracy and loss for training and test data using
    the mean of each epoch.
    Loss is displayed in a regular line, accuracy in a dotted
    line.
    Training data is displayed in blue, test data in red.
    Parameters
    ----------
    train_losses : numpy.ndarray
    training losses
    train_accuracies : numpy.ndarray
    training accuracies
    test_losses : numpy.ndarray
    test losses
    test_accuracies : numpy.ndarray
    test accuracies
    """
    plt.figure()
    line1, = plt.plot(train_losses, "b-")
    line2, = plt.plot(test_losses, "r-")
    line3, = plt.plot(train_accuracies, "b:")
    line4, = plt.plot(test_accuracies, "r:")
    plt.xlabel("Training steps")
    plt.ylabel("Loss / Accuracy")
    plt.title(name)
    plt.legend((line1, line2, line3, line4), ("training loss", "test loss", "train accuracy", "test accuracy"))
    plt.show()

In [ ]:
visualization(train_loss_list0_0, train_accuracy_list0_0, test_loss_list0_0, test_accuracy_list0_0, name="Model 0_0")
visualization(train_loss_list1_0, train_accuracy_list1_0, test_loss_list1_0, test_accuracy_list1_0, name="Model 1_0")
visualization(train_loss_list2_0, train_accuracy_list2_0, test_loss_list2_0, test_accuracy_list2_0, name="Model 2_0")
visualization(train_loss_list3_0, train_accuracy_list3_0, test_loss_list3_0, test_accuracy_list3_0, name="Model 3_0")
visualization(train_loss_list4_0, train_accuracy_list4_0, test_loss_list4_0, test_accuracy_list4_0, name="Model 4_0")

visualization(train_loss_list0_1, train_accuracy_list0_1, test_loss_list0_1, test_accuracy_list0_1, name="Model 0_1")
visualization(train_loss_list1_1, train_accuracy_list1_1, test_loss_list1_1, test_accuracy_list1_1, name="Model 1_1")
visualization(train_loss_list2_1, train_accuracy_list2_1, test_loss_list2_1, test_accuracy_list2_1, name="Model 2_1")
visualization(train_loss_list3_1, train_accuracy_list3_1, test_loss_list3_1, test_accuracy_list3_1, name="Model 3_1")
visualization(train_loss_list4_1, train_accuracy_list4_1, test_loss_list4_1, test_accuracy_list4_1, name="Model 4_1")

visualization(train_loss_list0_2, train_accuracy_list0_2, test_loss_list0_2, test_accuracy_list0_2, name="Model 0_2")
visualization(train_loss_list1_2, train_accuracy_list1_2, test_loss_list1_2, test_accuracy_list1_2, name="Model 1_2")
visualization(train_loss_list2_2, train_accuracy_list2_2, test_loss_list2_2, test_accuracy_list2_2, name="Model 2_2")
visualization(train_loss_list3_2, train_accuracy_list3_2, test_loss_list3_2, test_accuracy_list3_2, name="Model 3_2")
visualization(train_loss_list4_2, train_accuracy_list4_2, test_loss_list4_2, test_accuracy_list4_2, name="Model 4_2")

